# Challenge 2 - ANOVA

In statistics, **Analysis of Variance (ANOVA)** is also used to analyze the differences among group means. The difference between t-test and ANOVA is the former is ued to compare two groups whereas the latter is used to compare three or more groups. [Read more about the difference between t-test and ANOVA](http://b.link/anova24).

From the ANOVA test, you receive two numbers. The first number is called the **F-value** which indicates whether your null-hypothesis can be rejected. The critical F-value that rejects the null-hypothesis varies according to the number of total subjects and the number of subject groups in your experiment. In [this table](http://b.link/eda14) you can find the critical values of the F distribution. **If you are confused by the massive F-distribution table, don't worry. Skip F-value for now and study it at a later time. In this challenge you only need to look at the p-value.**

The p-value is another number yielded by ANOVA which already takes the number of total subjects and the number of experiment groups into consideration. **Typically if your p-value is less than 0.05, you can declare the null-hypothesis is rejected.**

In this challenge, we want to understand whether there are significant differences among various types of pokemons' `Total` value, i.e. Grass vs Poison vs Fire vs Dragon... There are many types of pokemons which makes it a perfect use case for ANOVA.

In [1]:
# Import libraries
import pandas as pd
import scipy.stats as st

In [28]:
# Load the data:
pokemon = pd.read_csv('Pokemon.csv')
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


**To achieve our goal, we use three steps:**

1. **Extract the unique values of the pokemon types.**

1. **Select dataframes for each unique pokemon type.**

1. **Conduct ANOVA analysis across the pokemon types.**

#### First let's obtain the unique values of the pokemon types. These values should be extracted from Type 1 and Type 2 aggregated. Assign the unique values to a variable called `unique_types`.

*Hint: the correct number of unique types is 19 including `NaN`. You can disregard `NaN` in next step.*

In [3]:
unique_types1 = list(pokemon['Type 1'].unique())
print(len(unique_types1)) # you should see 19
unique_types1

18


['Grass',
 'Fire',
 'Water',
 'Bug',
 'Normal',
 'Poison',
 'Electric',
 'Ground',
 'Fairy',
 'Fighting',
 'Psychic',
 'Rock',
 'Ghost',
 'Ice',
 'Dragon',
 'Dark',
 'Steel',
 'Flying']

#### Second we will create a list named `pokemon_totals` to contain the `Total` values of each unique type of pokemons.

Why we use a list instead of a dictionary to store the pokemon `Total`? It's because ANOVA only tells us whether there is a significant difference of the group means but does not tell which group(s) are significantly different. Therefore, we don't need know which `Total` belongs to which pokemon type.

*Hints:*

* Loop through `unique_types` and append the selected type's `Total` to `pokemon_groups`.
* Skip the `NaN` value in `unique_types`. `NaN` is a `float` variable which you can find out by using `type()`. The valid pokemon type values are all of the `str` type.
* At the end, the length of your `pokemon_totals` should be 18.

In [7]:
pokemon_totals = []

for poketype in unique_types1:
    pokemon_totals.append(pokemon[pokemon['Type 1'] == poketype]['Total'].mean())
    
pokemon_totals

[421.14285714285717,
 458.0769230769231,
 430.45535714285717,
 378.92753623188406,
 401.68367346938777,
 399.14285714285717,
 443.40909090909093,
 437.5,
 413.1764705882353,
 416.44444444444446,
 475.94736842105266,
 453.75,
 439.5625,
 433.4583333333333,
 550.53125,
 445.741935483871,
 487.7037037037037,
 485.0]

In [58]:
pokemon['type count'] = pokemon.groupby('Type 1').cumcount()
pokemon_pivot = pokemon.pivot(index='type count', columns='Type 1', values='Total')
pokemon_pivot

Type 1,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
type count,,,,,,,,,,,,,,,,,,
0,195.0,525.0,300.0,320.0,323.0,305.0,309.0,580.0,310.0,318.0,300.0,455.0,251.0,288.0,310.0,300.0,510.0,314.0
1,205.0,405.0,420.0,485.0,483.0,455.0,405.0,580.0,405.0,405.0,450.0,580.0,349.0,438.0,400.0,390.0,610.0,405.0
2,395.0,430.0,600.0,325.0,218.0,305.0,534.0,245.0,500.0,525.0,265.0,250.0,479.0,275.0,500.0,495.0,465.0,530.0
3,195.0,330.0,490.0,465.0,245.0,405.0,634.0,535.0,600.0,625.0,405.0,450.0,579.0,365.0,590.0,385.0,380.0,630.0
4,205.0,500.0,590.0,330.0,405.0,505.0,634.0,NaN,435.0,320.0,320.0,330.0,253.0,505.0,328.0,355.0,480.0,320.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,314.0
108,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,405.0
109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530.0


#### Now we run ANOVA test on `pokemon_totals`.

*Hints:*

* To conduct ANOVA, you can use `scipy.stats.f_oneway()`. Here's the [reference](http://b.link/scipy44).

* What if `f_oneway` throws an error because it does not accept `pokemon_totals` as a list? The trick is to add a `*` in front of `pokemon_totals`, e.g. `stats.f_oneway(*pokemon_groups)`. This trick breaks the list and supplies each list item as a parameter for `f_oneway`.

In [66]:
st.f_oneway(pokemon_pivot['Bug'].dropna(), pokemon_pivot['Dark'].dropna(),pokemon_pivot['Dragon'].dropna(),pokemon_pivot['Electric'].dropna(),pokemon_pivot['Fairy'].dropna(),pokemon_pivot['Fighting'].dropna(),pokemon_pivot['Fire'].dropna(),pokemon_pivot['Flying'].dropna(),pokemon_pivot['Ghost'].dropna(),pokemon_pivot['Grass'].dropna(),pokemon_pivot['Ground'].dropna(),pokemon_pivot['Ice'].dropna(),pokemon_pivot['Normal'].dropna(),pokemon_pivot['Poison'].dropna(),pokemon_pivot['Psychic'].dropna(),pokemon_pivot['Rock'].dropna(),pokemon_pivot['Steel'].dropna(),pokemon_pivot['Water'].dropna())

F_onewayResult(statistic=4.638767481660551, pvalue=2.077215448842098e-09)

#### Interpret the ANOVA test result. Is the difference significant?

In [67]:
# yes, it is. the null hypothesis is to be rejected